In [2]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.metrics import confusion_matrix, classification_report

import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from textblob import TextBlob

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

from matplotlib import pyplot as plt
import seaborn as sn

In [3]:
manual_df = pd.read_excel('C:/Users/Dishan/Vessel tracking project/manually_labeled_lemmatized_data.xlsx')
manual_df

,Date,Title,Description,Risk
0,2023-07-26,"['podcast', ':', 'in', 'conversation', 'with',...","['this', 'latest', 'episode', 'of', 'the', 'se...",False
1,2023-07-26,"['china', 'merchant', 'port', 'and', 'cma', 'c...","['the', 'chinese', 'port', 'operator', 'china'...",False
2,2023-07-26,"['cma', 'cgm', 'liberty', 'is', 'first', 'lng-...","['the', '15,000', 'teu', 'cma', 'cgm', 'libert...",False
3,2023-07-26,"['cyan', 'renewables', 'buying', 'first', 'sov...","['singapore-based', 'offshore', 'wind', 'farm'...",False
4,2023-07-26,"['shanghai', 'international', 'trade', 'exceed...","['shanghai', ""'s"", 'total', 'import', 'and', '...",False
...,...,...,...,...
688,NaN,"['2', 'long', 'beach', 'port', 'terminal', 'cl...","['long', 'beach', ',', 'calif.', '-', 'a', 'pa...",True
689,NaN,"['coastal', 'cargo', 'lay', 'off', 'it', 'work...","['the', 'decision', 'wa', 'taken', 'to', 'prev...",True
690,NaN,"['port', 'of', 'seattle', 'cargo', 'operation'...","['the', 'disruption', 'come', 'after', 'simila...",True
691,NaN,"['la-lb', 'outlook', 'darkens', 'a', 'labor', ...","['the', 'port', 'of', 'los', 'angeles', '’', '...",True


In [25]:
# manual_df['Risk'] = manual_df['Risk'].astype(bool)

In [26]:
# manual_df['Risk'].value_counts()

False    662
True      31
Name: Risk, dtype: int64

In [30]:
# !pip install textblob

In [31]:
# nltk.download('omw-1.4')

# # Download necessary resources for NLTK
# nltk.download('punkt')
# nltk.download('wordnet')

# # Initialize the lemmatizer
# lemmatizer = WordNetLemmatizer()

# # Function to tokenize and lemmatize text
# def tokenize_and_lemmatize(text):
#     if isinstance(text, str):  # Check if the text is not NaN or missing
#         words = word_tokenize(text.lower())
#         lemmatized_words = [lemmatizer.lemmatize(word) for word in words]
#         return lemmatized_words
#     else:
#         return []

# # Tokenize and lemmatize the 'Title' and 'Description' columns
# manual_df['Title'] = manual_df['Title'].apply(tokenize_and_lemmatize)
# manual_df['Description'] = manual_df['Description'].apply(tokenize_and_lemmatize)

[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\Dishan\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Dishan\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Dishan\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [32]:
# manual_df

,Date,Title,Description,Risk
0,2023-07-26,"[podcast, :, in, conversation, with, nikeel, i...","[this, latest, episode, of, the, seatrade, mar...",False
1,2023-07-26,"[china, merchant, port, and, cma, cgm, strengt...","[the, chinese, port, operator, china, merchant...",False
2,2023-07-26,"[cma, cgm, liberty, is, first, lng-powered, bo...","[the, 15,000, teu, cma, cgm, liberty, ha, beco...",False
3,2023-07-26,"[cyan, renewables, buying, first, sov, from, d...","[singapore-based, offshore, wind, farm, vessel...",False
4,2023-07-26,"[shanghai, international, trade, exceeds, $, 2...","[shanghai, 's, total, import, and, export, val...",False
...,...,...,...,...
688,NaN,"[2, long, beach, port, terminal, close, amid, ...","[long, beach, ,, calif., -, a, pair, of, cargo...",True
689,NaN,"[coastal, cargo, lay, off, it, workforce, ,, s...","[the, decision, wa, taken, to, prevent, any, c...",True
690,NaN,"[port, of, seattle, cargo, operation, continue...","[the, disruption, come, after, similar, work, ...",True
691,NaN,"[la-lb, outlook, darkens, a, labor, unrest, br...","[the, port, of, los, angeles, ’, sale, pitch, ...",True


In [17]:
# output_file='manually_labeled_lemmatized_data.xlsx'
# manual_df.to_excel(output_file, index=False)

In [4]:
vec_tdidf = TfidfVectorizer()

In [5]:
X_title = manual_df['Title']
X_description = manual_df['Description']
y = manual_df['Risk']

In [6]:
# Assuming X_title and X_description are lists of text documents
X_title = [' '.join(document) for document in X_title]  # Convert lists to strings
X_description = [' '.join(document) for document in X_description]

In [7]:
X_title

["[ ' p o d c a s t ' ,   ' : ' ,   ' i n ' ,   ' c o n v e r s a t i o n ' ,   ' w i t h ' ,   ' n i k e e l ' ,   ' i d n a n i ' ,   ' f r o m ' ,   ' i m a r e s t ' ,   ' u a e ' ,   ' b r a n c h ' ]",
 "[ ' c h i n a ' ,   ' m e r c h a n t ' ,   ' p o r t ' ,   ' a n d ' ,   ' c m a ' ,   ' c g m ' ,   ' s t r e n g t h e n ' ,   ' c o l l a b o r a t i o n ' ]",
 "[ ' c m a ' ,   ' c g m ' ,   ' l i b e r t y ' ,   ' i s ' ,   ' f i r s t ' ,   ' l n g - p o w e r e d ' ,   ' b o x s h i p ' ,   ' t o ' ,   ' c a l l ' ,   ' i n ' ,   ' j a p a n ' ]",
 "[ ' c y a n ' ,   ' r e n e w a b l e s ' ,   ' b u y i n g ' ,   ' f i r s t ' ,   ' s o v ' ,   ' f r o m ' ,   ' d e m e ' ]",
 "[ ' s h a n g h a i ' ,   ' i n t e r n a t i o n a l ' ,   ' t r a d e ' ,   ' e x c e e d s ' ,   ' $ ' ,   ' 2 8 0 ' ,   ' b i l l i o n ' ,   ' i n ' ,   ' h 1 ' ]",
 "[ ' p o o r ' ,   ' s o c i o - e c o n o m i c ' ,   ' c o n d i t i o n ' ,   ' , ' ,   ' c l i m a t e ' ,   ' c h a n g e 

In [8]:
X_title = vec_tdidf.fit_transform(X_title)
X_description = vec_tdidf.fit_transform(X_description)
print(X_title)
print(X_description)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [38]:
# Split the data into training and testing sets
X_train_title, X_test_title, X_train_description, X_test_description, y_train, y_test = train_test_split(
    X_title, X_description, y, test_size=0.2, random_state=42
)

In [39]:
# Concatenate the vectors to create a feature matrix
X_train = np.hstack((X_train_title.toarray(), X_train_description.toarray()))
X_test = np.hstack((X_test_title.toarray(), X_test_description.toarray()))

In [43]:
! pip install tensorflow
! pip install tensorflow_hub
! pip install tensorflow_text

  Using cached protobuf-4.24.3-cp39-cp39-win_amd64.whl (430 kB)
  Using cached tensorboard-2.13.0-py3-none-any.whl (5.6 MB)
  Using cached keras-2.13.1-py3-none-any.whl (1.7 MB)
  Using cached tensorflow_estimator-2.13.0-py2.py3-none-any.whl (440 kB)
  Using cached tensorboard_data_server-0.7.1-py3-none-any.whl (2.4 kB)
  Using cached google_auth_oauthlib-1.0.0-py2.py3-none-any.whl (18 kB)
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.10.0
    Uninstalling tensorflow-estimator-2.10.0:
      Successfully uninstalled tensorflow-estimator-2.10.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-server 0.6.1
    Uninstalling tensorboard-data-server-0.6.1:
      Successfully uninstalled tensorboard-data-server-0.6.1
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.19.6
    Uninstalling protobuf-3.19.6:
      Successfully uninstalled protobuf-3.19.6
  Attempting 

  Using cached tensorflow_text-2.10.0-cp39-cp39-win_amd64.whl (5.0 MB)
  Using cached tensorflow-2.10.1-cp39-cp39-win_amd64.whl (455.9 MB)
  Using cached keras-2.10.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorboard-2.10.1-py3-none-any.whl (5.9 MB)
  Using cached tensorflow_estimator-2.10.0-py2.py3-none-any.whl (438 kB)
  Using cached protobuf-3.19.6-cp39-cp39-win_amd64.whl (895 kB)
  Using cached google_auth_oauthlib-0.4.6-py2.py3-none-any.whl (18 kB)
  Using cached tensorboard_data_server-0.6.1-py3-none-any.whl (2.4 kB)
  Attempting uninstall: keras
    Found existing installation: keras 2.13.1
    Uninstalling keras-2.13.1:
      Successfully uninstalled keras-2.13.1
  Attempting uninstall: tensorflow-estimator
    Found existing installation: tensorflow-estimator 2.13.0
    Uninstalling tensorflow-estimator-2.13.0:
      Successfully uninstalled tensorflow-estimator-2.13.0
  Attempting uninstall: tensorboard-data-server
    Found existing installation: tensorboard-data-serv

ERROR: Could not install packages due to an OSError: [WinError 5] Access is denied: 'C:\\Users\\Dishan\\anaconda3\\Lib\\site-packages\\tensorflow\\compiler\\tf2tensorrt\\_pywrap_py_utils.pyd'
Consider using the `--user` option or check the permissions.



In [44]:
bert_preprocess = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_preprocess/3")
bert_encoder = hub.KerasLayer("https://tfhub.dev/tensorflow/bert_en_uncased_L-12_H-768_A-12/4")

ModuleNotFoundError: No module named 'tensorflow_text'

In [ ]:
# Bert layers
text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
preprocessed_text = bert_preprocess(text_input)
outputs = bert_encoder(preprocessed_text)

# Neural network layers
l = tf.keras.layers.Dropout(0.1, name="dropout")(outputs['pooled_output'])
l = tf.keras.layers.Dense(1, activation='sigmoid', name="output")(l)

# Use inputs and outputs to construct a final model
model = tf.keras.Model(inputs=[text_input], outputs = [l])

In [ ]:
model.summary()

In [ ]:
METRICS = [
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall')
]

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=METRICS)

In [ ]:
model.fit(X_train, y_train, epochs=10)

In [ ]:
model.evaluate(X_test, y_test)

In [ ]:
 y_predicted = model.predict(X_test)
 y_predicted = y_predicted.flatten()

In [ ]:
y_predicted = np.where(y_predicted > 0.5, 1, 0)
y_predicted

In [ ]:
cm = confusion_matrix(y_test, y_predicted)
cm 

In [ ]:
sn.heatmap(cm, annot=True, fmt='d')
plt.xlabel('Predicted')
plt.ylabel('Truth')